In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
#Directories of the TCGA data
CLIN_PATH = '../data/clinical.project-TCGA-BRCA.2016-09-30T12_32_20.666404.json' #Patient data
DIST_PATH = '../data/dd5_5_for_TCGA.txt' # Distance metric
GENOME_PATH = '../data/BRCA.rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.data.txt' #Genome data

#### Clinical data of patients

In [3]:
# Clinical data of patients
big_df = pd.read_json(CLIN_PATH)

# Change lists to dictionaries to easily expand to columns in DataFrame
def dictify(df):
    df['diagnoses'] = df['diagnoses'][0]
    df['exposures'] = df['exposures'][0]
    return df

big_df = big_df.dropna().apply(dictify,axis=1)

In [4]:
# Expand dictionaries to columns in DataFrame
df_clin = pd.DataFrame(big_df['case_id']).join(big_df['demographic'].apply(pd.Series),how='outer',lsuffix='_case_id',rsuffix='_demo')\
            .join(big_df['diagnoses'].apply(pd.Series),how='outer',lsuffix = '_demo',rsuffix='_diag')\
            .join(big_df['exposures'].apply(pd.Series),how='outer',lsuffix = '_diag',rsuffix='_expo')

In [5]:
# Drop nulls rows and columns
df_clin.dropna(how='all', inplace = True)
df_clin.dropna(how='all', inplace = True, axis=1)

In [6]:
df_clin.head()

,case_id,demographic_id,ethnicity,gender,race,submitter_id_demo,updated_datetime_demo,year_of_birth,year_of_death,age_at_diagnosis,...,site_of_resection_or_biopsy,submitter_id_diag,tissue_or_organ_of_origin,tumor_grade,tumor_stage,updated_datetime_diag,vital_status,exposure_id,submitter_id,updated_datetime
0,b205c89f-af62-4186-acad-ed23d243fa98,55ae5a3b-e509-5fcd-98c9-c5007b448890,not hispanic or latino,female,white,TCGA-A2-A0YL_demographic,2016-09-02T19:00:43.039678-05:00,1962.0,NaN,17702.0,...,c50.9,TCGA-A2-A0YL_diagnosis,c50.9,not reported,stage iiia,2016-09-02T19:00:43.039678-05:00,alive,23b6a296-e149-5801-aa20-e144ef683abe,TCGA-A2-A0YL_exposure,2016-09-02T19:00:43.039678-05:00
1,70f34c5c-3671-44c8-9469-99f9786efec1,498e27fb-2201-524a-bdd1-86f4e53dc54b,not hispanic or latino,female,white,TCGA-D8-A1XR_demographic,2016-09-02T19:07:03.114741-05:00,1954.0,NaN,20488.0,...,c50.9,TCGA-D8-A1XR_diagnosis,c50.9,not reported,stage iib,2016-09-02T19:07:03.114741-05:00,alive,6dfa11dc-4266-57c9-b746-02a6c7bb9d23,TCGA-D8-A1XR_exposure,2016-09-02T19:07:03.114741-05:00
2,8986a141-eae7-4157-b695-02cc6fc3b071,8faa57d4-1199-53ce-a0ce-24d358841153,not hispanic or latino,female,white,TCGA-BH-A1ET_demographic,2016-09-02T18:57:52.265307-05:00,1944.0,2005.0,20425.0,...,c50.9,TCGA-BH-A1ET_diagnosis,c50.9,not reported,stage i,2016-09-02T18:57:52.265307-05:00,dead,e72e6b02-58be-539a-af7f-f2714f14899a,TCGA-BH-A1ET_exposure,2016-09-02T18:57:52.265307-05:00
3,7317e605-93ba-49a9-8743-20613b02767a,4394b44c-b51d-5b96-b62b-db7f49007b43,not hispanic or latino,female,white,TCGA-E2-A1LE_demographic,2016-09-02T19:09:17.163541-05:00,1937.0,2010.0,26239.0,...,c50.9,TCGA-E2-A1LE_diagnosis,c50.9,not reported,stage iiic,2016-09-02T19:09:17.163541-05:00,dead,4c456a08-ffda-507f-a3e3-d273e5c49864,TCGA-E2-A1LE_exposure,2016-09-02T19:09:17.163541-05:00
4,7e1673f8-5758-4963-8804-d5e39f06205b,eebe9198-74c8-5996-bf88-c56d5fd6832f,not hispanic or latino,female,asian,TCGA-C8-A12V_demographic,2016-09-02T19:02:30.147198-05:00,1955.0,NaN,20346.0,...,c50.9,TCGA-C8-A12V_diagnosis,c50.9,not reported,stage iia,2016-09-02T19:02:30.147198-05:00,alive,dd5fbf53-64a4-5bc0-8ca7-ae05abbcbbac,TCGA-C8-A12V_exposure,2016-09-02T19:02:30.147198-05:00


#### Distances matrix

In [7]:
#Creating DataFrame from the distance-correlation matrix data
df_distances = pd.read_csv(DIST_PATH,sep="\t")

In [8]:
df_distances.head()

,TCGA.3C.AAAU.0.Dis,TCGA.3C.AALI.0.Dis,TCGA.3C.AALJ.0.Dis,TCGA.3C.AALK.0.Dis,TCGA.4H.AAAK.0.Dis,TCGA.5L.AAT0.0.Dis,TCGA.5L.AAT1.0.Dis,TCGA.5T.A9QA.0.Dis,TCGA.A1.A0SB.0.Dis,TCGA.A1.A0SD.0.Dis,...,TCGA.UL.AAZ6.0.Dis,TCGA.UU.A93S.0.Dis,TCGA.V7.A7HQ.0.Dis,TCGA.W8.A86G.0.Dis,TCGA.WT.AB41.0.Dis,TCGA.WT.AB44.0.Dis,TCGA.XX.A899.0.Dis,TCGA.XX.A89A.0.Dis,TCGA.Z7.A8R5.0.Dis,TCGA.Z7.A8R6.0.Dis
TCGA.3C.AAAU.0.Dis,0.000000,0.120880,0.105801,0.109540,0.118014,0.116020,0.138326,0.139012,0.164870,0.109789,...,0.130849,0.177643,0.202318,0.118387,0.204810,0.172970,0.131472,0.147361,0.160945,0.102312
TCGA.3C.AALI.0.Dis,0.120880,0.000000,0.086734,0.086734,0.105240,0.102997,0.095894,0.142688,0.183251,0.114400,...,0.117577,0.143124,0.172721,0.125615,0.158141,0.134712,0.116518,0.119447,0.129105,0.101502
TCGA.3C.AALJ.0.Dis,0.105801,0.086734,0.000000,0.072653,0.091719,0.082123,0.096455,0.141006,0.181756,0.102686,...,0.108979,0.147735,0.151162,0.113216,0.138139,0.120631,0.100941,0.103558,0.112343,0.083557
TCGA.3C.AALK.0.Dis,0.109540,0.086734,0.072653,0.000000,0.040750,0.040376,0.062932,0.144121,0.129292,0.053960,...,0.117827,0.158452,0.146738,0.062870,0.165742,0.102312,0.060814,0.073400,0.080130,0.070721
TCGA.4H.AAAK.0.Dis,0.118014,0.105240,0.091719,0.040750,0.000000,0.037074,0.065612,0.152471,0.113901,0.054209,...,0.143560,0.180884,0.172347,0.065300,0.187800,0.112343,0.059381,0.069163,0.097015,0.078198


#### Genome data

In [9]:
#Creating DataFrame from the Genome data
#Note please change the first two words of the text file: "Hybridization REF" to "Hybridization_REF"
with open(GENOME_PATH) as f:
    row = [row.split()[0:] for row in f]
df_genome = pd.DataFrame(row[1:],columns=row[0])

In [10]:
df_genome.head(n=3)
#Row 0 as it has no information.

,Hybridization_REF,TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALJ-01A-31R-A41B-07,TCGA-3C-AALK-01A-11R-A41B-07,TCGA-4H-AAAK-01A-12R-A41B-07,TCGA-5L-AAT0-01A-12R-A41B-07,TCGA-5L-AAT1-01A-12R-A41B-07,TCGA-5T-A9QA-01A-11R-A41B-07,TCGA-A1-A0SB-01A-11R-A144-07,...,TCGA-UL-AAZ6-01A-11R-A41B-07,TCGA-UU-A93S-01A-21R-A41B-07,TCGA-V7-A7HQ-01A-11R-A33J-07,TCGA-W8-A86G-01A-21R-A36F-07,TCGA-WT-AB41-01A-11R-A41B-07,TCGA-WT-AB44-01A-11R-A41B-07,TCGA-XX-A899-01A-11R-A36F-07,TCGA-XX-A89A-01A-11R-A36F-07,TCGA-Z7-A8R5-01A-42R-A41B-07,TCGA-Z7-A8R6-01A-11R-A41B-07
0,gene_id,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,...,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count
1,?|100130426,0.0000,0.0000,0.9066,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,?|100133144,16.3644,9.2659,11.6228,12.0894,6.8468,3.9889,0.0000,1.4644,15.3396,...,0.3992,4.3126,0.0000,5.5624,0.0000,0.0000,14.3858,22.3240,2.2638,6.8865


#### Removing data of the patients that are not in the three datasets

In [80]:
#We know that the label "TCGA-XX-XXXX" is a reference for each patient
#We take as a reference the file with less patients, that is to say: distances matrix data

#Taking df_distances as reference to match the other datasets 
ref_distance = [val.replace(".","-")[0:12] for val in df_distances.columns.tolist()]
print("Number of references: " + str(len(ref_distance)))

Number of references: 1093


In [81]:
#Getting the references for clinical data
ref_clinical = [val[0:12] for val in df_clin['submitter_id_demo'].astype(str).tolist()]
print("Number of references: " + str(len(ref_clinical)))

#Getting the references that are not in the matrix data (References that need to be removed)
print(set(ref_clinical)-set(ref_distance))

Number of references: 1097
set(['TCGA-AR-A0U1', 'TCGA-C8-A9FZ', 'TCGA-AC-A5EI', 'TCGA-A7-A0DC'])


In [83]:
#Getting the references for genome data
ref_genome = [val[0:12]for val in df_genome.columns.tolist()[1:]]
print("Number of references: " + str(len(ref_genome)))

#Getting the references that are not in the matrix data (References that need to be removed)
print(set(ref_genome)-set(ref_distance))

Number of references: 1212
set([])


In [90]:
#Since we saw that there are no extra references, we know then that there are duplicates.
#Getting the duplicated data
duplicates = [x for n, x in enumerate(ref_genome) if x in ref_genome[:n]]
print("There are " + str(len(duplicates)) + " duplicated references in genome data")

There are 119 duplicates of references in genome data


In [121]:
#These are the full name of the duplicates
duplicates_full = [i for e in duplicates for i in df_genome.columns.tolist() if e in i]

#Displaying just 2 duplicated references
duplicates_full[0:4]

['TCGA-A7-A0CE-11A-21R-A089-07',
 'TCGA-A7-A0CE-01A-11R-A00Z-07',
 'TCGA-A7-A0CH-11A-32R-A089-07',
 'TCGA-A7-A0CH-01A-21R-A00Z-07']

In [124]:
df_genome[[duplicates_full[0],duplicates_full[1]]].head()

,TCGA-A7-A0CE-11A-21R-A089-07,TCGA-A7-A0CE-01A-11R-A00Z-07
0,normalized_count,normalized_count
1,0.0000,0.0000
2,4.3333,11.0789
3,3.9206,3.4986
4,78.9238,130.2890
